In [56]:
import pandas as pd
import pickle
# Dosya yolları
train_path = 'twitter-veri-seti/train.csv'
test_path = 'twitter-veri-seti/test.csv'
validation_path = 'twitter-veri-seti/valid.csv'

# Verileri yükleyelim
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
validation_df = pd.read_csv(validation_path)

# Eğitim ve doğrulama verilerini birleştirelim

# İlk birkaç satırı görelim
print(train_df.head())
print(test_df.head())


        id                                               text  label
0  26418.0  Gerçekten sizin hikayelerinizi izleyerek mi ye...      0
1  14473.0            @USER Çoook çok bi baklava bi sen zaten      0
2  16107.0  1) Sn. DÜKEL; Atatürk'ün, Karma E. M. ile başl...      0
3  45908.0  Konfederasyonumuzun Aile ve Sosyal Politikalar...      0
4  12878.0  Hakemler tarih yazıyorlar / 9 kişiye karşı 3-2...      1
        id                                               text  label
0  41993.0    @USER Sayın başkanım bu şekilde devam inşallah👏      0
1  23000.0  Herkes gevşekliği kadar duyar kasıyor,hayat bö...      0
2  42478.0  Olgun ilişkisi olan arkadaş size en güzel hedi...      0
3  21748.0  @USER @USER Burada atıp tutacağına o kötü koşu...      1
4  13607.0   @USER İşte o onur dediğin sende yok sorun o işte      1


In [57]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Tokenizer oluştur ve eğit
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])

# Metinleri sekanslara çevir ve pad et
max_len = 50
X_train_seq = tokenizer.texts_to_sequences(train_df['text'])
X_test_seq = tokenizer.texts_to_sequences(test_df['text'])
X_val_seq = tokenizer.texts_to_sequences(validation_df['text'])

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

# Etiketleri encode et
le = LabelEncoder()
y_train = le.fit_transform(train_df['label'])
y_test = le.transform(test_df['label'])
y_val = le.transform(validation_df['label'])


In [58]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN,GRU, Dense, Dropout
from keras.regularizers import l2

# Modeli tanımla
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(GRU(128, return_sequences=False, kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 50, 50)            6659950   
                                                                 
 gru_7 (GRU)                 (None, 128)               69120     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 6729199 (25.67 MB)
Trainable params: 6729199 (25.67 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
from keras.callbacks import EarlyStopping

# EarlyStopping callback oluştur
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Modeli eğit
history = model.fit(X_train_pad, y_train, epochs=1, batch_size=32, validation_data=(X_val_pad, y_val), callbacks=[early_stopping])

# Test seti üzerinde değerlendirme
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')


277/277 [==============================] - 3s 9ms/step - loss: 0.2368 - accuracy: 0.9112
Test Accuracy: 91.12%


In [60]:
def predict_offensiveness(text):
    seq = tokenizer.texts_to_sequences([text])
    pad_seq = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(pad_seq)
    return 1 if pred >= 0.5 else 0

In [65]:
model.save('/content/offensive_language_rnn_model.h5')

# Tokenizer'ı kaydet
with open('/content/tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Etiket kodlayıcıyı kaydet
with open('/content/label_encoder.pkl', 'wb') as handle:
    pickle.dump(le, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
result=0.0
for i in range(len(validation_df["text"])):
  result += predict_offensiveness(validation_df["text"][i]) == validation_df["label"][i]

In [63]:
result / len(validation_df["text"])

0.8559225512528473